In [1]:
import sys
import os
sys.path.append(os.path.abspath('../src'))
from foe_dataset2 import FOEDataset
import torch
from matplotlib import pyplot as plt
import numpy as np
%matplotlib widget

real_path = '../datasets/Finger/FOESamples'
synth_path = '../datasets/Finger/synfin/v1'

dataset = FOEDataset(real_path, n_classes=256, patch_size=27, synth=False, subset="all")
# img = plt.imread(os.path.join(real_path,"Good","110.bmp"))
# fg = np.loadtxt(os.path.join(real_path,"Good","110.fg"),skiprows=1)
# gt = dataset.read_gt(os.path.join(real_path,"Good","110.gt"))
# plt.figure()
# dataset.plot_grid(img,dataset.extract_grid_data(img, fg, gt))

dataset2 = FOEDataset(synth_path, n_classes=256, patch_size=27, synth=True)
# img = plt.imread(os.path.join(synth_path,"200.png"))
# fg = np.loadtxt(os.path.join(synth_path,"200.fg"))
# gt = dataset2.read_gt(os.path.join(synth_path,"200.gt"))
# plt.figure()
# dataset.plot_grid(img,dataset2.extract_grid_data(img, fg, gt))

../datasets/Finger/FOESamples/Bad/00.bmp


/home/csigb/app/env/lib/python3.9/site-packages/torchvision/transforms/functional.py:74: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:141.)
  img = torch.from_numpy(pic.transpose((2, 0, 1))).contiguous()


../datasets/Finger/FOESamples/Bad/01.bmp
../datasets/Finger/FOESamples/Bad/02.bmp
../datasets/Finger/FOESamples/Bad/03.bmp
../datasets/Finger/FOESamples/Bad/04.bmp
../datasets/Finger/FOESamples/Bad/05.bmp
../datasets/Finger/FOESamples/Bad/06.bmp
../datasets/Finger/FOESamples/Bad/07.bmp
../datasets/Finger/FOESamples/Bad/08.bmp
../datasets/Finger/FOESamples/Bad/09.bmp
../datasets/Finger/FOESamples/Bad/10.bmp
../datasets/Finger/FOESamples/Bad/11.bmp
../datasets/Finger/FOESamples/Bad/12.bmp
../datasets/Finger/FOESamples/Bad/13.bmp
../datasets/Finger/FOESamples/Bad/14.bmp
../datasets/Finger/FOESamples/Bad/15.bmp
../datasets/Finger/FOESamples/Bad/16.bmp
../datasets/Finger/FOESamples/Bad/17.bmp
../datasets/Finger/FOESamples/Bad/18.bmp
../datasets/Finger/FOESamples/Bad/19.bmp
../datasets/Finger/FOESamples/Bad/20.bmp
../datasets/Finger/FOESamples/Bad/21.bmp
../datasets/Finger/FOESamples/Bad/22.bmp
../datasets/Finger/FOESamples/Bad/23.bmp
../datasets/Finger/FOESamples/Bad/24.bmp
../datasets/Fing

In [2]:
class NeuralNetwork(torch.nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()

        self.cnn_layers = torch.nn.Sequential(
            # Defining a 2D convolution layer
            torch.nn.BatchNorm2d(1),
            torch.nn.Conv2d(1, 4, kernel_size=3, stride=1, padding=1),
            torch.nn.BatchNorm2d(4),
            torch.nn.ReLU(inplace=True),
            torch.nn.MaxPool2d(kernel_size=2, stride=2),
            torch.nn.Dropout(p=0.3),
            # Defining another 2D convolution layer
            torch.nn.Conv2d(4, 4, kernel_size=3, stride=1, padding=1),
            torch.nn.BatchNorm2d(4),
            torch.nn.ReLU(inplace=True),
            torch.nn.Dropout(p=0.3),
            # Defining another 2D convolution layer
            torch.nn.Conv2d(4, 4, kernel_size=3, stride=1, padding=1),
            torch.nn.BatchNorm2d(4),
            torch.nn.ReLU(inplace=True),
            torch.nn.Dropout(p=0.3),
            # Defining another 2D convolution layer
            torch.nn.Conv2d(4, 4, kernel_size=3, stride=1, padding=1),
            torch.nn.BatchNorm2d(4),
            torch.nn.ReLU(inplace=True),
            torch.nn.Dropout(p=0.3),
        )

        self.linear_layers = torch.nn.Sequential(
            torch.nn.Linear(4 * 13 * 13, 512),
            torch.nn.Dropout(p=0.3),
            torch.nn.Linear(512, 256),
            torch.nn.Dropout(p=0.3),
            torch.nn.Linear(256, 256),
            torch.nn.Dropout(p=0.3),
            torch.nn.Linear(256, 256),
            torch.nn.Dropout(p=0.3),
            torch.nn.Linear(256, 1),
        )
    
    # Defining the forward pass    
    def forward(self, x):
        x = self.cnn_layers(x)
        x = x.view(x.size(0), -1)
#         print(x.shape)
        x = self.linear_layers(x)
        return x

    def init_weights(self,m):
        if type(m) == torch.nn.Linear:
            torch.nn.init.kaiming_uniform_(m.weight, nonlinearity='leaky_relu')
            m.bias.data.fill_(0.01)

In [3]:
def train(dataloader, model, loss_fn, optimizer, scheduler):
    train_loss, correct = 0, 0
    i = 0
    for batch, (X, y) in enumerate(dataloader):
        y = y.view((-1, 1))
#         X = X.view((X.shape[0],1,X.shape[1],X.shape[2]))
#         X = X.repeat(1,3,1,1)
        X, y = X.to(device), y.to(device)

        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y.float())
        train_loss += loss.item()
        correct += (pred.argmax(1) == y).type(torch.float).sum().item()
        i+=1

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

#         if batch % 100 == 0:
#             loss, current = loss.item(), batch * len(X)
#             print(f"loss: {loss:>7f}  [{current:>5d}]")
    scheduler.step()
    train_loss /= len(dataloader)
    correct /= len(dataloader.dataset)
    print(f"Training Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {train_loss:>8f}")
    return[train_loss, correct]

In [4]:
def test(dataloader, model):
    model.eval()
    test_loss, correct = 0, 0
    predd, yy = [], []
    with torch.no_grad():
        for X, y in dataloader:
            y = y.view(-1, 1)
#             X = X.view((X.shape[0],1,X.shape[1],X.shape[2]))
#             X = X.repeat(1,3,1,1)
            X, y = X.to(device), y.to(device)
            pred = model(X)
            predd.append(pred)
            yy.append(y)
            loss = loss_fn(pred, y.float())
            test_loss += loss.item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= len(dataloader)
    correct /= len(dataloader.dataset)
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f}")
    return[test_loss, correct, predd, yy]

In [11]:
# # import torchvision.models as models

# device = "cuda" if torch.cuda.is_available() else "cpu"
# print("Using {} device".format(device))
# loss_fn = torch.nn.MSELoss()

# N = len(dataset)
# batch_size = 128
# train_size = round(N*0.9)
# val_size = round(N*0.1)
# test_size = round(N*0.1)
# training_data, test_data = torch.utils.data.random_split(dataset, [train_size, test_size], generator=torch.Generator().manual_seed(42))
# training_data2, test_data2 = torch.utils.data.random_split(dataset2, [len(dataset2), 0], generator=torch.Generator().manual_seed(42))

# folds = 1
# test_dataloader =  torch.utils.data.DataLoader(test_data, batch_size=batch_size)
# for f in range(folds):
#     model = NeuralNetwork().to(device)
# #     model = models.alexnet(pretrained=True)
# #     new_classifier = torch.nn.Sequential(*list(model.classifier.children())+[torch.nn.ReLU(inplace=True),torch.nn.Linear(1000, 1),])
# #     model.classifier = new_classifier
# #     if torch.cuda.is_available():
# #         model.cuda()
# #     optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
#     optimizer = torch.optim.SGD(model.parameters(), lr=0.001)
#     scheduler = torch.optim.lr_scheduler.CyclicLR(optimizer, 0.0001, 0.1)
#     train_dataloader =  torch.utils.data.DataLoader(training_data, batch_size=batch_size)
#     val_dataloader =  torch.utils.data.DataLoader(test_data, batch_size=batch_size)

#     epochs = 500
#     train_res = []
#     test_res = []
#     for t in range(epochs):
#         print(f"Epoch {t+1}\n-------------------------------")
#         train_ret = train(train_dataloader, model, loss_fn, optimizer, scheduler)
#         test_ret = test(val_dataloader, model)
#         train_res.append(train_ret)
#         test_res.append(test_ret)
#     print("Done!")

# for f in range(folds):
#     model2 = NeuralNetwork().to(device)
# #     model2 = models.alexnet(pretrained=True)
# #     new_classifier = torch.nn.Sequential(*list(model2.classifier.children())+[torch.nn.ReLU(inplace=True),torch.nn.Linear(1000, 1),])
# #     model2.classifier = new_classifier
# #     if torch.cuda.is_available():
# #         model2.cuda()
# #     optimizer = torch.optim.Adam(model2.parameters(), lr=0.001)
#     optimizer = torch.optim.SGD(model2.parameters(), lr=0.001)
#     scheduler = torch.optim.lr_scheduler.CyclicLR(optimizer, 0.0001, 0.01)
#     train_dataloader =  torch.utils.data.DataLoader(training_data2, batch_size=batch_size)
#     val_dataloader =  torch.utils.data.DataLoader(test_data, batch_size=batch_size)

#     epochs = 500
#     train_res2 = []
#     test_res2 = []
#     for t in range(epochs):
#         print(f"Epoch {t+1}\n-------------------------------")
#         train_ret = train(train_dataloader, model2, loss_fn, optimizer, scheduler)
#         test_ret = test(val_dataloader, model2)
#         train_res2.append(train_ret)
#         test_res2.append(test_ret)
#     print("Done!")
    
# for f in range(folds):
#     model3 = NeuralNetwork().to(device)
# #     model2 = models.alexnet(pretrained=True)
# #     new_classifier = torch.nn.Sequential(*list(model2.classifier.children())+[torch.nn.ReLU(inplace=True),torch.nn.Linear(1000, 1),])
# #     model2.classifier = new_classifier
# #     if torch.cuda.is_available():
# #         model2.cuda()
# #     optimizer = torch.optim.Adam(model2.parameters(), lr=0.001)
#     optimizer = torch.optim.SGD(model3.parameters(), lr=0.001)
#     scheduler = torch.optim.lr_scheduler.CyclicLR(optimizer, 0.0001, 0.01)
#     train_dataloader =  torch.utils.data.DataLoader(torch.utils.data.ConcatDataset([training_data, training_data2]), batch_size=batch_size)
#     val_dataloader =  torch.utils.data.DataLoader(test_data, batch_size=batch_size)

#     epochs = 500
#     train_res3 = []
#     test_res3 = []
for t in range(500):
    print(f"Epoch {t+1}\n-------------------------------")
    train_ret = train(train_dataloader, model3, loss_fn, optimizer, scheduler)
    test_ret = test(val_dataloader, model3)
    train_res3.append(train_ret)
    test_res3.append(test_ret)
print("Done!")


Epoch 1
-------------------------------
Training Error: 
 Accuracy: 26.7%, Avg loss: 0.022452
Test Error: 
 Accuracy: 8.1%, Avg loss: 0.055920
Epoch 2
-------------------------------
Training Error: 
 Accuracy: 26.7%, Avg loss: 0.022425
Test Error: 
 Accuracy: 8.1%, Avg loss: 0.059542
Epoch 3
-------------------------------
Training Error: 
 Accuracy: 26.7%, Avg loss: 0.022402
Test Error: 
 Accuracy: 8.1%, Avg loss: 0.061535
Epoch 4
-------------------------------
Training Error: 
 Accuracy: 26.7%, Avg loss: 0.022386
Test Error: 
 Accuracy: 8.1%, Avg loss: 0.063295
Epoch 5
-------------------------------
Training Error: 
 Accuracy: 26.7%, Avg loss: 0.022372
Test Error: 
 Accuracy: 8.1%, Avg loss: 0.064841
Epoch 6
-------------------------------


KeyboardInterrupt: 

In [10]:
%matplotlib widget

from matplotlib import pyplot as plt
import numpy as np

plt.figure()
plt.plot(np.array(train_res)[:,0])
plt.plot(np.array(test_res)[:,0])
plt.figure()
plt.plot(np.array(train_res2)[:,0])
plt.plot(np.array(test_res2)[:,0])
plt.figure()
plt.plot(np.array(train_res3)[:,0])
plt.plot(np.array(test_res3)[:,0])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

<ipython-input-10-d4e31f8c949a>:11: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  plt.plot(np.array(test_res2)[:,0])


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

<ipython-input-10-d4e31f8c949a>:14: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  plt.plot(np.array(test_res3)[:,0])


In [15]:
print(np.array(train_res)[:,0].min())
print(np.array(test_res)[:,0].min())
print(np.array(train_res2)[:,0].min())
print(np.array(test_res2)[:,0].min())
print(np.array(train_res3)[:,0].min())
print(np.array(test_res3)[:,0].min())

0.03868118962873405
0.04196266026546558
0.021687816271861626
0.06593483418226243
0.02237168367403422
0.04735254059235255


<ipython-input-15-14d87a146fce>:4: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  print(np.array(test_res2)[:,0].min())
<ipython-input-15-14d87a146fce>:6: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  print(np.array(test_res3)[:,0].min())


In [8]:
train_dataloader =  torch.utils.data.DataLoader(training_data2, batch_size=batch_size)
test_loss, correct, predd, yy = test(train_dataloader, model)
# aaa = np.array([])
# bbb = np.array([])
# for p in predd: aaa = np.append(aaa, p.cpu().data.numpy().flatten())
# for y in yy: bbb = np.append(bbb, y.cpu().data.numpy().flatten())
# ccc = np.power(np.mean(np.power((aaa-bbb)*180%90,2)),0.5)
# print(aaa.shape, bbb.shape, test_loss, ccc)

print(predd[0].transpose(0,1).max())
print(predd[0].transpose(0,1).min())
print(torch.cat(yy).shape)
print(torch.cat(yy).max(), torch.cat(yy).min())
print(torch.cat(yy).mean(), torch.cat(yy).std())
print(torch.cat(yy).min())
print(torch.cat(predd).max())
print(torch.cat(predd).min())
plt.figure()
plt.imshow(dataset.data[:,4][3000])
print(dataset.data[:,3][3000])
print(torch.cat(yy)[3000])

Test Error: 
 Accuracy: 26.7%, Avg loss: 0.072242
tensor(0.7391, device='cuda:0')
tensor(-0.5993, device='cuda:0')
torch.Size([247560, 1])
tensor(0.4944, device='cuda:0', dtype=torch.float64) tensor(-0.5000, device='cuda:0', dtype=torch.float64)
tensor(-0.0045, device='cuda:0', dtype=torch.float64) tensor(0.3314, device='cuda:0', dtype=torch.float64)
tensor(-0.5000, device='cuda:0', dtype=torch.float64)
tensor(1.2152, device='cuda:0')
tensor(-0.9942, device='cuda:0')


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

TypeError: Invalid shape (1, 27, 27) for image data